Instalace balíků - potřeba pustit pouze pokud notebook pouštíte poprvé

In [ ]:
# using Pkg;
# Pkg.instantiate();

Načtení používaných balíků

In [1]:
using GeometricFlux;
using Flux;
using Flux: onecold, crossentropy, throttle, glorot_uniform, @epochs;
using JLD2;  # use v0.1.2
using SparseArrays;
using Statistics: mean;
using LightGraphs: SimpleGraphs, adjacency_matrix;

Načtení dat - dataset Cora

In [2]:
@load "data/cora_features.jld2" features;
@load "data/cora_labels.jld2" labels;
@load "data/cora_graph.jld2" g;

train_X = Float32.(features);  # dim: num_features * num_nodes
train_y = Float32.(labels);  # dim: target_catg * num_nodes

adj_mat = Matrix{Float32}(adjacency_matrix(g));

Nastavení parametrů modelu
- Šířka skryté vrstvy
- Počet výstupních tříd
- Počet trénovacích epoch

In [3]:
hidden_layer_width = 16;
num_classes = 7;
epochs = 20;

Definice modelu pomocí metod balíčku `GeometricFlux.jl`
- Jedna vrstva GCN šířky `hidden_layer_width` s aktivační funkcí ReLU
- Dropout
- Druhá vrstva GCN šířky `num_classes` s lineární aktivací
- Softmax funkce

In [ ]:
model = Chain(
    GCNConv(adj_mat, size(train_X, 1) => hidden_layer_width, relu),
    Dropout(0.5),
    GCNConv(adj_mat, hidden_layer_width => num_classes),
    softmax
);

parameters = Flux.params(model);

Alternativně si mohu GCN vrstvy definovat sám

In [4]:
W1 = Float32.(glorot_uniform(hidden_layer_width, size(train_X, 1)));
b1 = Float32.(glorot_uniform(hidden_layer_width))
function GCN1(X::AbstractMatrix)
    L = normalized_laplacian(adj_mat, eltype(X); selfloop = true);
    return relu.(W1 * X * L .+ b1)
end

W2 = Float32.(glorot_uniform(num_classes, hidden_layer_width));
b2 = Float32.(glorot_uniform(num_classes))
function GCN2(X::AbstractMatrix)
    L = normalized_laplacian(adj_mat, eltype(X); selfloop = true);
    return W2 * X * L .+ b2
end

model = Chain(
    GCN1,
    Dropout(0.5),
    GCN2,
    softmax
);

parameters = Flux.params(W1, b1, W2, b2);

Definice ztrátové funkce - cross-entropy. Jako průběžnou míru budeme ukazovat přesnost na trénovacích datech.

In [5]:
loss(x, y) = crossentropy(model(x), y);
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y));

Trénujeme pomocí metody ADAM s `η = 0.05`.

In [6]:
train_data = [(train_X, train_y)];
opt = ADAM(0.05);
evalcb() = @show(accuracy(train_X, train_y));

@epochs epochs Flux.train!(loss, parameters, train_data, opt, cb=throttle(evalcb, 10));

┌ Info: Epoch 1
└ @ Main /home/marekdedic/.julia/packages/Flux/05b38/src/optimise/train.jl:114


accuracy(train_X, train_y) = 0.17540620384047267


┌ Info: Epoch 2
└ @ Main /home/marekdedic/.julia/packages/Flux/05b38/src/optimise/train.jl:114


accuracy(train_X, train_y) = 0.2891432791728213


┌ Info: Epoch 3
└ @ Main /home/marekdedic/.julia/packages/Flux/05b38/src/optimise/train.jl:114


...

accuracy(train_X, train_y) = 0.5214180206794683


┌ Info: Epoch 19
└ @ Main /home/marekdedic/.julia/packages/Flux/05b38/src/optimise/train.jl:114


accuracy(train_X, train_y) = 0.5273264401772526


┌ Info: Epoch 20
└ @ Main /home/marekdedic/.julia/packages/Flux/05b38/src/optimise/train.jl:114


accuracy(train_X, train_y) = 0.53397341211226


Kód je modifikací příkladů balíčku [GeometricFlux.jl](https://github.com/yuehhua/GeometricFlux.jl).